In [1]:
import torch
import sympy as sp
import cvxpy as cp
import numpy as np
sp.init_printing(use_unicode=True)

# Second non-numbered equation

In [2]:
# Parameters (theta is 2D, requires gradients)
theta = torch.tensor([1.0, 2.0], requires_grad=True)

# Define state s_t(theta) ∈ R^2
s1 = theta[0] ** 2 + theta[1]
s2 = torch.sin(theta[0])
s_t = torch.stack([s1, s2])

# Define action a_t(theta) ∈ R^1
a_t = torch.tensor([theta[0] * theta[1]])


# Reward function
def r_fn(s, a):
    return s[0] * s[1] + a[0] ** 2


# Reward r(s, a) scalar
r = r_fn(s_t, a_t)

# Automatic gradient via PyTorch
r.backward()
grad_autodiff = theta.grad.clone()
theta.grad.zero_()

print("Automatic gradient:", grad_autodiff)

dr_ds = torch.autograd.functional.jacobian(lambda s: r_fn(s, a_t), s_t)
dr_da = torch.autograd.functional.jacobian(lambda a: r_fn(s_t, a), a_t)
ds_dtheta = torch.autograd.functional.jacobian(lambda th: torch.stack([th[0] ** 2 + th[1], torch.sin(th[0])]), theta)
da_dtheta = torch.autograd.functional.jacobian(lambda th: torch.tensor([th[0] * th[1]]), theta)
grad_formula = dr_ds @ ds_dtheta + dr_da @ da_dtheta

print("Formula gradient:", grad_formula)

Automatic gradient: tensor([3.3038, 0.8415])
Formula gradient: tensor([3.3038, 0.8415])


/tmp/ipykernel_3222994/2670516345.py:10: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  a_t = torch.tensor([theta[0] * theta[1]])


# Eq. 11

In [3]:
# Input action
a_i = torch.tensor([1.5], requires_grad=True)

# a_s as function of a
a_s = torch.sin(a_i)

# Next state s_{i+1} as function of a_s
s_next = a_s ** 2


# Reward depends on both a_s and s_next
def r_fn(a_s, s_next):
    return a_s * s_next + a_s ** 3  # scalar


r = r_fn(a_s, s_next)

# ---- Ground truth (autograd) ----
r.backward()
grad_autodiff = a_i.grad.clone()
a_i.grad.zero_()
print("Autograd total gradient:", grad_autodiff)

# ---- Branch decomposition ----
# 1. Direct path: dr/da_s holding s_next constant
dr_da_s_direct = torch.autograd.functional.jacobian(
    lambda a_s: r_fn(a_s, s_next.detach()), a_s
)

# 2. Indirect path: dr/ds_next * ds_next/da_s
dr_ds = torch.autograd.functional.jacobian(lambda s: r_fn(a_s.detach(), s), s_next)
ds_da_s = torch.autograd.functional.jacobian(lambda a_s: a_s ** 2, a_s)
indirect = dr_ds @ ds_da_s

# 3. Combine
dr_da_s_total = dr_da_s_direct + indirect

# 4. Multiply by da_s/da
da_s_da = torch.autograd.functional.jacobian(torch.sin, a_i)
grad_formula = dr_da_s_total @ da_s_da

print("Formula (split paths):", grad_formula)

Autograd total gradient: tensor([0.4223])
Formula (split paths): tensor([[0.4223]])


# Eq 13 & 14

In [37]:
def cvxpy_jacobian(problem, parameter, variable):
    jacobian = np.zeros((variable.shape[0], parameter.shape[0]))  # 2x2 matrix for ∂cp_safe_action/∂cp_action
    for i in range(jacobian.shape[1]):
        parameter.delta = np.zeros(jacobian.shape[1])
        parameter.delta[i] = 1.0
        problem.derivative()
        jacobian[:, i] = variable.delta
    return jacobian

In [62]:
# Define (12)
cp_action = cp.Parameter(2)
safe_action_center = np.zeros(2)
safe_action_generator = np.eye(2)

cp_safe_action = cp.Variable(2)
weights = cp.Variable(2)

objective = cp.Minimize(cp.sum_squares(cp_action - cp_safe_action))

pos_constraint = cp_safe_action - safe_action_center == safe_action_generator @ weights
size_constraint = cp.norm(weights, "inf") <= 1
constraints = [pos_constraint, size_constraint]

problem = cp.Problem(objective, constraints)

a_s_boundary = np.array([1, 0])
v = np.array([1, 0])
eps = 1e-5
for t in np.linspace(0, 1, 10):
    a_u = a_s_boundary + t * v
    cp_action.value = a_u
    problem.solve(requires_grad=True)
    # Verify (13)
    assert np.allclose(a_s_boundary, cp_safe_action.value, atol=1e-3)


    jacobian = cvxpy_jacobian(problem, cp_action, cp_safe_action)

    upstream_gradient = np.ones_like(cp_safe_action.value)  # dr/da_s
    assert np.allclose(upstream_gradient @ jacobian @ v, 0.0, atol=1e-5)

# Eq 15

In [75]:
# Check (15) for a bunch of random values

for d in range(2, 10):
    cp_action = cp.Parameter(d)
    safe_action_center = np.zeros(d)
    safe_action_generator = np.random.uniform(size=(d, 2*d)) * 0.5

    cp_safe_action = cp.Variable(d)
    weights = cp.Variable(2*d)

    objective = cp.Minimize(cp.sum_squares(cp_action - cp_safe_action))

    pos_constraint = cp_safe_action - safe_action_center == safe_action_generator @ weights
    size_constraint = cp.norm(weights, "inf") <= 1
    constraints = [pos_constraint, size_constraint]

    problem = cp.Problem(objective, constraints)

    for val in np.random.uniform(size=(10, d)):
        cp_action.value = val
        problem.solve(requires_grad=True)
        jacobian = cvxpy_jacobian(problem, cp_action, cp_safe_action)

        if np.allclose(val, cp_safe_action.value, atol=1e-3):
            # already safe
            assert d == np.linalg.matrix_rank(jacobian, tol=1e-4)
        else:
            assert d > np.linalg.matrix_rank(jacobian, tol=1e-4)

# Eq 17

In [5]:
a = torch.tensor([1.0, 2.0], requires_grad=True)
s = torch.tensor([0.1, -0.2])
c_d = 0.5


def l_r(a_s, s):
    return 0.5 * torch.sum((a_s - s) ** 2)


a_s = torch.tanh(a)
loss = l_r(a_s, s) + c_d * torch.sum((a_s - a) ** 2)
loss.backward()
grad_autodiff = a.grad.clone()
a.grad.zero_()

# Manual gradient
dl_r_da = torch.autograd.functional.jacobian(lambda a_: l_r(torch.tanh(a_), s), a)
da_s_da = torch.diag(1 - torch.tanh(a) ** 2)  # Jacobian of tanh
grad_manual = dl_r_da + 2 * c_d * (a_s - a).T @ (da_s_da - torch.eye(2))

print("Automatic gradient:", grad_autodiff)
print("Manual gradient:", grad_manual)

Automatic gradient: tensor([0.4161, 1.0450])
Manual gradient: tensor([0.4161, 1.0450], grad_fn=<AddBackward0>)


/tmp/ipykernel_3222994/3143576541.py:19: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4421.)
  grad_manual = dl_r_da + 2 * c_d * (a_s - a).T @ (da_s_da - torch.eye(2))


# Eq 32

In [6]:

# Define symbols
lambda_a, lambda_As, lambda_A = sp.symbols('lambda_a lambda_As lambda_A', real=True, positive=True)

# Original mapping
omega_tanh = sp.tanh(lambda_a / lambda_As) / sp.tanh(lambda_A / lambda_As)

# SymPy derivative
domega_dlambda_a_sympy = sp.diff(omega_tanh, lambda_a)

# Claimed formula
domega_dlambda_a_claimed = (1 - sp.tanh(lambda_a / lambda_As) ** 2) / (lambda_As * sp.tanh(lambda_A / lambda_As))

print("Difference after simplification:", sp.simplify(domega_dlambda_a_sympy - domega_dlambda_a_claimed))

Difference after simplification: 0


# Proof of Theorem 1

In [7]:
#Symbols
d = 3
n = 4
a = sp.Matrix(d, 1, lambda i, _: sp.symbols(f'a{i}'))
a_s = sp.Matrix(d, 1, lambda i, _: sp.Function(f'a_s_{i}')(*a))
c = sp.Matrix(sp.symbols(f'c0:{d}'))
G = sp.Matrix(sp.symbols(f'G0:{d * n}')).reshape(d, n)
gamma = sp.Matrix(n, 1, lambda i, _: sp.Function(f'gamma_{i}')(*a))
z = sp.Matrix.vstack(a_s, gamma)
Q = sp.BlockMatrix([
    [2 * sp.eye(d), sp.zeros(d, n)],
    [sp.zeros(n, d), sp.zeros(n, n)]
]).as_explicit()
q = -2 * sp.Matrix.vstack(a, sp.zeros(n, 1))
A = sp.Matrix.hstack(sp.eye(d), -G)
b = c
K = sp.BlockMatrix([
    [sp.zeros(n, d), sp.eye(n)],
    [sp.zeros(n, d), -sp.eye(n)]
]).as_explicit()
h = sp.ones(2 * n, 1)

dual_lambda = sp.Matrix(2 * n, 1, lambda i, _: sp.Function(f'lambda_{i}')(*a))
dual_nu = sp.Matrix(d, 1, lambda i, _: sp.Function(f'nu_{i}')(*a))

d_z = z.jacobian(a)
d_lambda = dual_lambda.jacobian(a)
d_nu = dual_nu.jacobian(a)
d_a_s = a_s.jacobian(a)
d_gamma = gamma.jacobian(a)

In [8]:
# Equation 44
objective_44 = (a - a_s).T * (a - a_s)
equalities_44 = a_s - (c + G * gamma)
inequalities_44 = sp.Matrix.vstack(gamma, -gamma) - sp.ones(2 * n, 1)

# Equation 45
objective_45 = (1 / 2) * z.T * Q * z + q.T * z
equalities_45 = A * z - b
inequalities_45 = K * z - h

print("Difference of objectives (45) - (44):", sp.simplify(objective_45 - objective_44))
print("Difference of equalities (45) - (44):", sp.simplify(equalities_45 - equalities_44))
print("Difference of inequalities (45) - (44):", sp.simplify(inequalities_45 - inequalities_44))

Difference of objectives (45) - (44): Matrix([[-a0**2 - a1**2 - a2**2]])
Difference of equalities (45) - (44): Matrix([[0], [0], [0]])
Difference of inequalities (45) - (44): Matrix([[0], [0], [0], [0], [0], [0], [0], [0]])


In [9]:
# Equation 53
A_53 = sp.BlockMatrix([
    [Q, K.T, A.T],
    [sp.diag(*dual_lambda) * K, sp.diag(*(K * z - h)), sp.zeros(2 * n, d)],
    [A, sp.zeros(d, 2 * n), sp.zeros(d, d)]
]).as_explicit()
x_53 = sp.Matrix.vstack(d_z, d_lambda, d_nu)
b_53 = sp.BlockMatrix([
    [2 * sp.eye(d)],
    [sp.zeros(3 * n + d, d)],
]).as_explicit()

# Equation 54
A_54 = sp.BlockMatrix([
    [2 * sp.eye(d), sp.zeros(d, n), sp.zeros(d, 2 * n), sp.eye(d)],
    [sp.zeros(n, d), sp.zeros(n, n), sp.BlockMatrix([sp.eye(n), -sp.eye(n)]), -G.T],
    [sp.zeros(2 * n, d), sp.diag(*dual_lambda) * sp.BlockMatrix([[sp.eye(n)], [-sp.eye(n)]]).as_explicit(),
     sp.BlockMatrix([[sp.diag(*gamma) - sp.eye(n), sp.zeros(n, n)],
                     [sp.zeros(n, n), -sp.diag(*gamma) - sp.eye(n)]]).as_explicit(), sp.zeros(2 * n, d)],
    [sp.eye(d), -G, sp.zeros(d, 2 * n), sp.zeros(d, d)]
]).as_explicit()
x_54 = sp.Matrix.vstack(d_a_s, d_gamma, d_lambda, d_nu)
b_54 = b_53

print("Difference of A (53) - (54):", sp.simplify(A_53 - A_54))
print("Difference of x (53) - (54):", sp.simplify(x_53 - x_54))
print("Difference of b (53) - (54):", sp.simplify(b_53 - b_54))
print("Shape compatibility:", (A_54 * x_53).shape == b_53.shape)

Difference of A (53) - (54): Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0

In [10]:
# Equations (55) - (58)
lhs_55 = 2 * d_a_s + d_nu
rhs_55 = 2 * sp.eye(d)
lhs_56 = sp.BlockMatrix([sp.eye(n), -sp.eye(n)]).as_explicit() * d_lambda - G.T * d_nu
rhs_56 = sp.zeros(n, d)
lhs_57 = sp.diag(*dual_lambda) * sp.BlockMatrix([[sp.eye(n)], [-sp.eye(n)]]).as_explicit() * d_gamma + sp.BlockMatrix(
    [[sp.diag(*gamma) - sp.eye(n), sp.zeros(n, n)],
     [sp.zeros(n, n), -sp.diag(*gamma) - sp.eye(n)]]).as_explicit() * d_lambda
rhs_57 = sp.zeros(2 * n, d)
lhs_58 = d_a_s -G * d_gamma
rhs_58 = sp.zeros(d, d)
print("Difference of A_54 * x_54  - lhs_55-lhs_58", sp.simplify(A_54 * x_54 - sp.Matrix.vstack(lhs_55, lhs_56, lhs_57, lhs_58)))
print("Difference of b_54  - rhs_55-rhs_58", sp.simplify(b_54 - sp.Matrix.vstack(rhs_55, rhs_56, rhs_57, rhs_58)))

Difference of A_54 * x_54  - lhs_55-lhs_58 Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]])
Difference of b_54  - rhs_55-rhs_58 Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [11]:
# Equation 59
eq_58 = sp.Eq(lhs_58, rhs_58)
eq_59 = sp.Eq(d_a_s, G*d_gamma)
print("Difference of (58) and (59)", sp.simplify((lhs_58 - rhs_58) - (eq_59.lhs - eq_59.rhs)))

Difference of (58) and (59) Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [12]:
# Equation 60
eq_60 = sp.Eq(d_nu, 2*sp.eye(d)-2*G*d_gamma)
subs_dict_59 = {eq_59.lhs[i, j]: eq_59.rhs[i, j] for i in range(eq_59.lhs.rows) for j in range(eq_59.lhs.cols)}
print("Difference of (55)+(59) and (60)", sp.simplify((lhs_55 - rhs_55).subs(subs_dict_59) - (eq_60.lhs - eq_60.rhs)))

Difference of (55)+(59) and (60) Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [13]:
# Equation 61
eq_61 = sp.Eq(0.5 * sp.BlockMatrix([sp.eye(n), -sp.eye(n)]).as_explicit() * d_lambda, G.T - G.T*G*d_gamma)
subs_dict_60 = {eq_60.lhs[i, j]: eq_60.rhs[i, j] for i in range(eq_60.lhs.rows) for j in range(eq_60.lhs.cols)}
print("Difference of (56)+(60) and (61)", sp.simplify((lhs_56 - rhs_56).subs(subs_dict_60) - 2*(eq_61.lhs - eq_61.rhs)))

Difference of (56)+(60) and (61) Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [14]:
# Equation 63
I_a = [0, 1]
I_i = list(range(2,2*n))
subs_dict_63 = {gamma[i, 0]: 1 for i in I_a} | {dual_lambda[i, 0]: 0 for i in I_i}

In [15]:
eq_63 = sp.Eq(sp.BlockMatrix([[sp.diag(*dual_lambda[I_a,0]) * d_gamma[I_a, :]],
                              [sp.diag(*sp.Matrix.vstack(gamma[2:, :]- sp.ones(2, 1), -gamma - sp.ones(4, 1)))*d_lambda[I_i, :] ]]).as_explicit(),sp.zeros(2*n,d))
print("Difference of Indices + (57) and (63)", sp.simplify((lhs_57 - rhs_57).subs(subs_dict_63) - (eq_63.lhs - eq_63.rhs).subs(subs_dict_63)))

Difference of Indices + (57) and (63) Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [39]:
# Trying an example to verify (71)
cp_action = cp.Parameter(2)
cp_action.value = np.array([2.0, 0.0])

safe_action_center = np.zeros(2)
safe_action_generator = np.eye(2)

cp_safe_action = cp.Variable(2)
weights = cp.Variable(2)

objective = cp.Minimize(cp.sum_squares(cp_action - cp_safe_action))

pos_constraint = cp_safe_action - safe_action_center == safe_action_generator @ weights
size_constraint = cp.norm(weights, "inf") <= 1
constraints = [pos_constraint, size_constraint]

problem = cp.Problem(objective, constraints)
problem.solve(requires_grad=True)

inactive_generators = safe_action_generator[:, 1:2]
expected_jacobian = inactive_generators @ np.linalg.inv(inactive_generators.T @ inactive_generators) @ inactive_generators.T
numerical_jacobian = cvxpy_jacobian(problem, cp_action, cp_safe_action)

assert np.allclose(expected_jacobian, numerical_jacobian, atol=1e-6)

In [126]:
# Check (71) for a bunch of random values, fails sometimes due to numerics or being close to the border/corners where we are no actually differentiable

for d in range(2, 10):
    cp_action = cp.Parameter(d)
    safe_action_center = np.zeros(d)
    safe_action_generator = np.random.uniform(size=(d, 2*d)) * 0.5

    cp_safe_action = cp.Variable(d)
    weights = cp.Variable(2*d)

    objective = cp.Minimize(cp.sum_squares(cp_action - cp_safe_action))

    pos_constraint = cp_safe_action - safe_action_center == safe_action_generator @ weights
    size_constraint = cp.norm(weights, "inf") <= 1
    constraints = [pos_constraint, size_constraint]

    problem = cp.Problem(objective, constraints)

    for val in np.random.uniform(size=(10, d)):
        cp_action.value = val
        problem.solve(requires_grad=True)
        numerical_jacobian = cvxpy_jacobian(problem, cp_action, cp_safe_action)
        inactive_generators = safe_action_generator[:, np.abs(weights.value) < 0.99]
        expected_jacobian = inactive_generators @ np.linalg.pinv(inactive_generators.T @ inactive_generators) @ inactive_generators.T
        try:
            assert np.allclose(expected_jacobian, numerical_jacobian, atol=1e-4)
        except AssertionError:
            print(f"Safe: {np.allclose(cp_action.value, cp_safe_action.value, atol=1e-3)}, dimension {d}")
            print(np.round(expected_jacobian, 3))
            print(np.round(numerical_jacobian, 3))
            print(np.round(inactive_generators, 3)) # If empty we are on a corner
            print(np.round(cp_action.value - cp_safe_action.value, 3)) # if very small we are barely differentiable as we are close to the boundary, which makes it numerically unstable
            raise AssertionError

optimal
Safe: False, dimension 5
[[ 0.124 -0.157  0.116  0.142  0.225]
 [-0.157  0.971  0.008  0.04   0.038]
 [ 0.116  0.008  0.539  0.47  -0.118]
 [ 0.142  0.04   0.47   0.44   0.061]
 [ 0.225  0.038 -0.118  0.061  0.925]]
[[ 0.956 -0.012 -0.131  0.158 -0.016]
 [-0.012  0.997 -0.035  0.042 -0.004]
 [-0.131 -0.035  0.613  0.466 -0.047]
 [ 0.158  0.042  0.466  0.441  0.056]
 [-0.016 -0.004 -0.047  0.056  0.994]]
[[0.139 0.028 0.203]
 [0.146 0.478 0.02 ]
 [0.148 0.347 0.463]
 [0.216 0.338 0.471]
 [0.43  0.05  0.268]]
[ 0.049  0.013  0.145 -0.174  0.017]


AssertionError: 

In [122]:
cp_action.value - cp_safe_action.value

array([-0.00422075,  0.0051669 ])